# 前処理

## データ確認

In [1]:
import pandas as pd
df = pd.read_csv("data/cyberguard_table.csv")


In [206]:
df.tail()

,営業活動結果,B to B企業,B to C企業,オーナー企業（非上場）,自社HPがある企業,増収企業,大幅減収企業,減収企業,連続増収増益の企業（2020年度と2019年度比較）,社員一人当たりの売上が高い企業,...,WiFiサービスの契約数,オンラインストレージサービスの契約数,サポートサービスの契約数,ハイエンドVPNサービスの契約数,リーズナブルVPNサービスの契約数,資本金（千円）,従業員数,事業所数,最新期業績売上高（百万円）,企業の総合評価点数
10638,失注,1.0,1.0,1.0,0.0,0,0.0,1,0.0,1.0,...,0,0,0,0,0,50000以下,5-20,10以下,100-500,50-60
10639,失注,1.0,1.0,1.0,1.0,0,0.0,1,0.0,0.0,...,0,0,0,0,4,50000以下,300以上,10-50,1000-10000,50-60
10640,失注,1.0,1.0,1.0,1.0,0,0.0,1,0.0,1.0,...,0,0,0,0,0,100000-300000,100-300,10以下,10000以上,50-60
10641,受注,1.0,1.0,1.0,1.0,0,0.0,1,0.0,1.0,...,1,0,1,0,0,NaN,NaN,NaN,NaN,NaN
10642,受注,1.0,1.0,1.0,1.0,0,1.0,1,0.0,0.0,...,0,0,0,0,0,50000以下,5-20,10以下,100-500,40-50


## 欠損値処理

In [38]:
df.isnull().sum()


営業活動結果                           0
B to B企業                      1668
B to C企業                      1668
オーナー企業（非上場）                   1668
自社HPがある企業                      735
増収企業                             0
大幅減収企業                        1668
減収企業                             0
連続増収増益の企業（2020年度と2019年度比較）     735
社員一人当たりの売上が高い企業                735
グループ従業員数が1000人以上の企業            735
従業員が増えた企業                        0
従業員が大幅に増えた企業                  1668
従業員が減った企業                     1668
従業員が大幅に減った企業                  1668
特定の地方にのみ事業展開をしている企業            735
ユーザ単位所掌事業部                     272
大分類名                          1381
中分類漢字名称                       1381
小分類漢字名称                       1381
産業分類名                         1381
インターネット回線サービス契約数                 0
光電話サービスの契約数                      0
WiFiサービスの契約数                     0
オンラインストレージサービスの契約数               0
サポートサービスの契約数                     0
ハイエンドVPNサービスの契約数                 0
リーズナブルVPNサービスの契約数                0
資本金（千円）             

1668, 1381, 735と同じ数が多く現れており、1つの企業で欠損値があるときは沢山あり、ないときはないとはっきり分かれていると考えられる。

そのため、欠損値が1つでもある行を全て削除してもデータ数への影響は少ないと判断し、欠損値がある行は削除することにした。

In [207]:
X = df.dropna(subset=["大分類名", "B to B企業", "社員一人当たりの売上が高い企業", "ユーザ単位所掌事業部"])
X.shape

(7346, 33)

7346/10642 残っているので、良いのかな。

## 特徴量の整理

### A 営業活動結果

まず、受注、失注を1,0に変換する。

In [217]:
X = X.replace(["受注", "失注"], [1, 0])

### B~P 〇〇な企業かどうか

#### BtoB, BtoC, オーナーについて

- BtoBとBtoCには差はないと判断したため、オーナー企業のみ残す。

In [225]:
X.groupby(["B to B企業"],dropna=False, as_index=False)["営業活動結果"].mean()

,B to B企業,営業活動結果
0,0.0,0.161972
1,1.0,0.149432


In [226]:
X.groupby(["B to C企業"],dropna=False, as_index=False)["営業活動結果"].mean()

,B to C企業,営業活動結果
0,0.0,0.152690
1,1.0,0.155308


In [227]:
X.groupby(["オーナー企業（非上場）"],dropna=False, as_index=False)["営業活動結果"].mean()

,オーナー企業（非上場）,営業活動結果
0,0.0,0.122369
1,1.0,0.172793


#### 減収、増収について

- 大幅減収と減収は差はないと判断したので、増収のみ残す。

In [237]:
X.groupby(["増収企業"],dropna=False, as_index=False)["営業活動結果"].mean()

,増収企業,営業活動結果
0,0,0.161583
1,1,0.137931


In [230]:
X.groupby(["大幅減収企業"],dropna=False, as_index=False)["営業活動結果"].mean()

,大幅減収企業,営業活動結果
0,0.0,0.153180
1,1.0,0.158287


In [231]:
X.groupby(["減収企業"],dropna=False, as_index=False)["営業活動結果"].mean()

,減収企業,営業活動結果
0,0,0.156216
1,1,0.149151


In [238]:
X.groupby(["連続増収増益の企業（2020年度と2019年度比較）"],dropna=False, as_index=False)["営業活動結果"].mean()

,連続増収増益の企業（2020年度と2019年度比較）,営業活動結果
0,0.0,0.154878
1,1.0,0.104167


In [306]:
X.replace(["5以下", "5-20", "20-50", "50-100", "100-300", "300以上"], ["50以下", "50以下", "50以下", "50以上", "50以上", "50以上"])\
    .groupby(["従業員数"],dropna=False, as_index=False)["営業活動結果"].mean()

,従業員数,営業活動結果
0,50以上,0.094611
1,50以下,0.184053


### Q ユーザ単位所掌事業部

- one-hotエンコーディングを行う

In [257]:
X.groupby(["ユーザ単位所掌事業部"],dropna=False, as_index=False)["営業活動結果"].mean()

,ユーザ単位所掌事業部,営業活動結果
0,1_東京事業部,0.203256
1,2_神奈川事業部,0.103527
2,3_千葉事業部,0.221860
3,4_埼玉事業部,0.197002
4,5_宮城事業部,0.160947
5,6_北海道事業部,0.060083
6,7_品川本部,0.043478


In [308]:
# pd.get_dummies(X,columns=["ユーザ単位所掌事業部"])

### R~U 大分類名や産業分類名

- 大分類名はone-hotエンコーディングを行う。

- 中分類漢字名称、小分類漢字名称、産業分類名 は分類が細かすぎるため除外することにした。

In [273]:
X.groupby(["大分類名"],dropna=False, as_index=False).count()

,大分類名,営業活動結果,B to B企業,B to C企業,オーナー企業（非上場）,自社HPがある企業,増収企業,大幅減収企業,減収企業,連続増収増益の企業（2020年度と2019年度比較）,...,WiFiサービスの契約数,オンラインストレージサービスの契約数,サポートサービスの契約数,ハイエンドVPNサービスの契約数,リーズナブルVPNサービスの契約数,資本金（千円）,従業員数,事業所数,最新期業績売上高（百万円）,企業の総合評価点数
0,サービス業,1710,1710,1710,1710,1710,1710,1710,1710,1710,...,1710,1710,1710,1710,1710,1710,1710,1710,1710,1710
1,不動産業,428,428,428,428,428,428,428,428,428,...,428,428,428,428,428,428,428,428,428,428
2,公務（他に分類されないもの）,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
3,卸売・小売業，飲食店,2007,2007,2007,2007,2007,2007,2007,2007,2007,...,2007,2007,2007,2007,2007,2007,2007,2007,2007,2007
4,建設業,1173,1173,1173,1173,1173,1173,1173,1173,1173,...,1173,1173,1173,1173,1173,1173,1173,1173,1173,1173
5,林業，狩猟業,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
6,漁業,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
7,製造業,1299,1299,1299,1299,1299,1299,1299,1299,1299,...,1299,1299,1299,1299,1299,1299,1299,1299,1299,1299
8,農業,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
9,運輸・通信業,536,536,536,536,536,536,536,536,536,...,536,536,536,536,536,536,536,536,536,536


### V~AB 契約数

- 標準化を行う。

In [286]:
from sklearn.preprocessing import StandardScaler
scaling_columns = ['インターネット回線サービス契約数', '光電話サービスの契約数', 'WiFiサービスの契約数',
                   'オンラインストレージサービスの契約数', 'サポートサービスの契約数', 'ハイエンドVPNサービスの契約数',
                   'リーズナブルVPNサービスの契約数']
sc = StandardScaler().fit(X[scaling_columns])
X_scaled = pd.DataFrame(sc.transform(X[scaling_columns]),
             columns=scaling_columns, index=X.index)
X[scaling_columns] = X_scaled
X.head()

,営業活動結果,B to B企業,B to C企業,オーナー企業（非上場）,自社HPがある企業,増収企業,大幅減収企業,減収企業,連続増収増益の企業（2020年度と2019年度比較）,社員一人当たりの売上が高い企業,...,WiFiサービスの契約数,オンラインストレージサービスの契約数,サポートサービスの契約数,ハイエンドVPNサービスの契約数,リーズナブルVPNサービスの契約数,資本金（千円）,従業員数,事業所数,最新期業績売上高（百万円）,企業の総合評価点数
0,0,1.0,0.0,1.0,0.0,1,0.0,0,1.0,1.0,...,-0.166273,0.598141,-0.175411,-0.091571,-0.101510,50000以下,5以下,10以下,10-100,40-50
1,0,1.0,0.0,1.0,0.0,1,0.0,0,0.0,1.0,...,-0.166273,-0.217819,-0.175411,-0.091571,-0.101510,50000以下,20-50,10以下,500-1000,50-60
2,0,1.0,0.0,1.0,0.0,1,0.0,0,0.0,1.0,...,-0.166273,-0.217819,-0.175411,-0.091571,-0.101510,50000以下,5-20,10以下,100-500,50-60
4,0,1.0,0.0,1.0,0.0,1,0.0,0,0.0,1.0,...,-0.166273,0.598141,-0.175411,-0.091571,0.111019,50000以下,20-50,10以下,100-500,40-50
5,0,1.0,0.0,1.0,0.0,1,0.0,0,0.0,1.0,...,-0.166273,-0.217819,-0.175411,-0.091571,-0.101510,50000-100000,100-300,10以下,1000-10000,50-60


### AC~AH 資本金、従業員数、企業の総合評価など

#### 資本金

In [295]:
X.replace(["50000以下", "50000-100000","100000-300000", "300000以上"], ["50000以下", "50000以上", "50000以上", "50000以上"]).groupby(["資本金（千円）"],dropna=False, as_index=False)["営業活動結果"].mean()

,資本金（千円）,営業活動結果
0,50000以上,0.101187
1,50000以下,0.170163


#### 従業員数

In [307]:
X.replace(["5以下", "5-20", "20-50", "50-100", "100-300", "300以上"], ["50以下", "50以下", "50以下", "50以上", "50以上", "50以上"])\
    .groupby(["従業員数"],dropna=False, as_index=False)["営業活動結果"].mean()

,従業員数,営業活動結果
0,50以上,0.094611
1,50以下,0.184053


#### 事業所数

In [296]:
X.replace(["10以下", "10-50", "50以上"], ["10以下", "10以上", "10以上"])\
    .groupby(["事業所数"],dropna=False, as_index=False)["営業活動結果"].mean()

,事業所数,営業活動結果
0,10以上,0.084615
1,10以下,0.158805


#### 最新期業績売上高

In [301]:
X.replace(["10以下", "10-100", "100-500", "500-1000", "1000-10000", "10000以上"], ["1000以下", "1000以下", "1000以下", "1000以下", "1000以上", "1000以上"])\
    .groupby(["最新期業績売上高（百万円）"], dropna=False, as_index=False)["営業活動結果"].mean()


,最新期業績売上高（百万円）,営業活動結果
0,1000以上,0.103279
1,1000以下,0.193730


#### 企業の総合評価点数

In [302]:
X.replace(["30以下", "30-40", "40-50", "50-60", "60-70", "70以上"], ["50以下", "50以下", "50以下", "50以上", "50以上", "50以上"])\
    .groupby(["企業の総合評価点数"],dropna=False, as_index=False)["営業活動結果"].mean()

,企業の総合評価点数,営業活動結果
0,50以上,0.118792
1,50以下,0.193023


## 前処理実装

今まででまとめたことを、実装する。

In [315]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
scaling_columns = ['インターネット回線サービス契約数', '光電話サービスの契約数', 'WiFiサービスの契約数',
                   'オンラインストレージサービスの契約数', 'サポートサービスの契約数', 'ハイエンドVPNサービスの契約数',
                   'リーズナブルVPNサービスの契約数']

df = pd.read_csv("data/cyberguard_table.csv")

df_cleaned = df.dropna(subset=["大分類名", "B to B企業", "社員一人当たりの売上が高い企業", "ユーザ単位所掌事業部"])\
    .replace(["受注", "失注"], [1, 0])\
    .drop(["B to B企業", "B to C企業", "減収企業", "大幅減収企業"], axis=1)

df_cleaned["資本金（千円）"] = df_cleaned["資本金（千円）"].replace(
    ["50000以下", "50000-100000","100000-300000", "300000以上"], ["50000以下", "50000以上", "50000以上", "50000以上"])
df_cleaned["従業員数"] = df_cleaned["従業員数"].replace(
    ["5以下", "5-20", "20-50", "50-100", "100-300", "300以上"], ["50以下", "50以下", "50以下", "50以上", "50以上", "50以上"])
df_cleaned["事業所数"] = df_cleaned["事業所数"].replace(
    ["10以下", "10-50", "50以上"], ["10以下", "10以上", "10以上"])
df_cleaned["最新期業績売上高（百万円）"] = df_cleaned["最新期業績売上高（百万円）"].replace(
    ["10以下", "10-100", "100-500", "500-1000", "1000-10000", "10000以上"], ["1000以下", "1000以下", "1000以下", "1000以下", "1000以上", "1000以上"])
df_cleaned["企業の総合評価点数"] = df_cleaned["企業の総合評価点数"].replace(["30以下", "30-40", "40-50", "50-60", "60-70", "70以上"], ["50以下", "50以下", "50以下", "50以上", "50以上", "50以上"])

df_cleaned= pd.get_dummies(df_cleaned,columns=["ユーザ単位所掌事業部","資本金（千円）","従業員数","事業所数","最新期業績売上高（百万円）","企業の総合評価点数"])\
    .drop(["資本金（千円）_50000以下", "従業員数_50以下", "事業所数_10以下", "最新期業績売上高（百万円）_1000以下", "企業の総合評価点数_50以下"], axis=1)

sc = StandardScaler().fit(X[scaling_columns])
df_scaled = pd.DataFrame(sc.transform(df_cleaned[scaling_columns]),
                          columns=scaling_columns, index=df_cleaned.index)
df_cleaned[scaling_columns] = df_scaled
df_cleaned.head()


,営業活動結果,オーナー企業（非上場）,自社HPがある企業,増収企業,連続増収増益の企業（2020年度と2019年度比較）,社員一人当たりの売上が高い企業,グループ従業員数が1000人以上の企業,従業員が増えた企業,従業員が大幅に増えた企業,従業員が減った企業,...,ユーザ単位所掌事業部_3_千葉事業部,ユーザ単位所掌事業部_4_埼玉事業部,ユーザ単位所掌事業部_5_宮城事業部,ユーザ単位所掌事業部_6_北海道事業部,ユーザ単位所掌事業部_7_品川本部,資本金（千円）_50000以上,従業員数_50以上,事業所数_10以上,最新期業績売上高（百万円）_1000以上,企業の総合評価点数_50以上
0,0,1.0,0.0,1,1.0,1.0,0.0,0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,0,1.0,0.0,1,0.0,1.0,0.0,0,0.0,1.0,...,0,0,0,1,0,0,0,0,0,1
2,0,1.0,0.0,1,0.0,1.0,0.0,0,0.0,1.0,...,0,0,0,1,0,0,0,0,0,1
4,0,1.0,0.0,1,0.0,1.0,0.0,0,0.0,1.0,...,0,0,0,1,0,0,0,0,0,0
5,0,1.0,0.0,1,0.0,1.0,0.0,0,0.0,1.0,...,0,1,0,0,0,1,1,0,1,1
